In [2]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import torch
import copy
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import Tensor

In [3]:
train = pd.read_csv('dataAll.csv')
test = pd.read_csv('testAll.csv')
train_data=train.iloc[:,:30].applymap(lambda x: x+126.23)
train_loc = train.iloc[:,30:]

x_test = test.applymap(lambda x: x+126.23)
x_test = torch.from_numpy(
       x_test.values.astype(np.float32))

In [4]:
train_feat = torch.from_numpy(
        train_data.values.astype(np.float32))
train_labels = torch.from_numpy(
        train_loc.values.astype(np.float32))

In [5]:
num_train = int(0.8 * train_feat.shape[0]) # 划分训练样本和验证集样本
indices = np.arange( train_feat.shape[0])
np.random.shuffle(indices)  # shuffle 顺序
train_indices = indices[:num_train]
valid_indices = indices[num_train:]

train_features = train_feat[train_indices,:]
train_label = train_labels[train_indices,:]
valid_features  = train_feat[valid_indices,:]
valid_labels = train_labels[valid_indices,:]


In [6]:
feat_dim = train_data.shape[0]

In [7]:
def error(y_test,y_pre):
    y_test = np.array(y_test)
    y_pre = np.array(y_pre)
    m = y_test-y_pre
    m=np.square(m)
    m=m.sum(axis=1)
    m=np.sqrt(m)
    return m.sum()/len(y_test)

def get_model(feat_dim,output_dim = 2 ):
    net = nn.Sequential(
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim*4/5)),
        nn.BatchNorm1d(int(feat_dim*4/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*4/5),int(feat_dim*3/5)),
        nn.BatchNorm1d(int(feat_dim*3/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*3/5),int(feat_dim*2/5)),
        nn.BatchNorm1d(int(feat_dim*2/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*2/5),int(feat_dim/5)),
        nn.BatchNorm1d(int(feat_dim/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim/5), output_dim)
        )
    return net

def get_data(x,y,batch_size,shuffle):
    dataset = TensorDataset(x,y)
    return DataLoader(dataset, batch_size, shuffle=shuffle,num_workers=4)

def train_model(model,x_train,y_train,x_valid,y_valid,epochs,batch_size,lr,weight_decay,use_gpu):
    
    if use_gpu:
        model = model.cuda()
    train_data = get_data(x_train,y_train,batch_size,True)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()
    #criterion = nn.CrossEntropyLoss()
    best_error = 0
    for e in  range(epochs):
        model.train()
        for data in train_data:
            x,y = data
            if use_gpu:
                x = x.cuda()
                y = y.cuda()
                
            out = model(x)
            loss = criterion(out, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if use_gpu:
                x_valid = x_valid.cuda()
        with torch.no_grad():
            output = model(x_valid)
        output = Tensor.cpu(output)
        out = output.numpy()
        print('epoch:{}'.format(e))
        error_i = error(out,y_valid)
        print('error:{}'.format(error_i)) 
        if e == 0:
            best_error = error_i
        elif error_i < best_error:
            best_model = copy.deepcopy(model)
    torch.save(best_model,"model_NN_master.t7")

def predict(x_test,use_gpu):
    net = torch.load('./model_NN_master.t7')
    if use_gpu:
        x_test = x_test.cuda()
    with torch.no_grad():
        a = Tensor.cpu(net(x_test)).numpy()
    #a = net(x_test).detach().numpy()
    #a = np.argmax(a,axis=1)
    #np.savetxt('NN_ans.txt',np.array(a),fmt='%f')
    #print(a)
    a = pd.DataFrame(a)
    a.to_csv('NN_base.csv',header=None)
    '''
    with open('./NN_master_ans.txt','w') as f:
        for i in a:
            f.write('{}'.format(i))
            f.write('\n')'''
    

In [ ]:
# 可以调整的超参
batch_size = 128
epochs = 40000
lr = 0.001
wd = 0
use_gpu = False
feat_dim = train_feat.shape[1]
net = get_model(feat_dim)
train_model(net, train_features, train_label, valid_features, valid_labels, epochs, 
            batch_size, lr, wd, use_gpu)
predict(x_test,use_gpu)

epoch:0
error:37.69037789052393
epoch:1
error:37.58692795924968
epoch:2
error:37.47698950921733
epoch:3
error:37.35023346539457
epoch:4
error:37.217269364489006
epoch:5
error:37.07077740944373
epoch:6
error:36.909433619016816
epoch:7
error:36.71758267302717
epoch:8
error:36.50266059589263
epoch:9
error:36.28588898771022
epoch:10
error:36.05718386157827
epoch:11
error:35.80273690168176
epoch:12
error:35.547792185478656
epoch:13
error:35.282894869825355
epoch:14
error:35.027136562095734
epoch:15
error:34.71421713292367
epoch:16
error:34.414115560316944
epoch:17
error:34.10220175048512
epoch:18
error:33.801069291720566
epoch:19
error:33.45310478654592
epoch:20
error:33.097246927554984
epoch:21
error:32.74895395375162
epoch:22
error:32.38204438874515
epoch:23
error:31.981676503880983
epoch:24
error:31.620426706015525
epoch:25
error:31.203716243531694
epoch:26
error:30.776643859152653
epoch:27
error:30.327372048835706
epoch:28
error:29.92785565572445
epoch:29
error:29.524966142464425
epoch:

epoch:241
error:2.170645769324062
epoch:242
error:2.062350767858587
epoch:243
error:2.0538142156169146
epoch:244
error:2.0171394298795278
epoch:245
error:1.9790664416235446
epoch:246
error:2.0060719321030076
epoch:247
error:1.9952968977401357
epoch:248
error:1.983451024064117
epoch:249
error:2.010200214262613
epoch:250
error:1.9452043264371766
epoch:251
error:2.0409761455975097
epoch:252
error:2.02933714399054
epoch:253
error:2.0046134050574063
epoch:254
error:1.9365089090798835
epoch:255
error:1.9292705975097024
epoch:256
error:2.0029555859880337
epoch:257
error:1.9784204006811934
epoch:258
error:1.904902804869421
epoch:259
error:1.885506258590718
epoch:260
error:1.9692900782260672
epoch:261
error:1.9191438149357212
epoch:262
error:1.8370940885753557
epoch:263
error:1.8578520425695342
epoch:264
error:1.9234961821838616
epoch:265
error:1.9077682199021668
epoch:266
error:1.9235863531391495
epoch:267
error:1.9185688369178526
epoch:268
error:2.3475946621321153
epoch:269
error:2.0967302211

epoch:477
error:1.6148648477926908
epoch:478
error:1.5922946313065978
epoch:479
error:1.5651226956662354
epoch:480
error:1.6162955813389392
epoch:481
error:1.5728173891292043
epoch:482
error:1.6793072343952133
epoch:483
error:1.6267489375303201
epoch:484
error:1.5815197548613358
epoch:485
error:1.5429586432729625
epoch:486
error:1.5714771423734637
epoch:487
error:1.5920850746381792
epoch:488
error:1.6895314521345408
epoch:489
error:1.559793923835705
epoch:490
error:1.659757981787678
epoch:491
error:1.5799623713918984
epoch:492
error:1.5830307105534445
epoch:493
error:1.6061166859536709
epoch:494
error:1.572004903025954
epoch:495
error:1.53198210603978
epoch:496
error:1.5769270370108344
epoch:497
error:1.5770022057931758
epoch:498
error:1.5551441977280078
epoch:499
error:1.5570951118814682
epoch:500
error:1.5196253688955368
epoch:501
error:1.5645169237144243
epoch:502
error:1.5319805268636804
epoch:503
error:1.5294914294954722
epoch:504
error:1.5862517560438227
epoch:505
error:1.5573100

epoch:713
error:1.6657012110385672
epoch:714
error:1.5988538971539457
epoch:715
error:1.5508816855999352
epoch:716
error:1.5346288051827297
epoch:717
error:1.5473943910191623
epoch:718
error:1.4953635389715394
epoch:719
error:1.5642235127951165
epoch:720
error:1.6925165687156372
epoch:721
error:1.5347756685599936
epoch:722
error:1.5981549538122575
epoch:723
error:1.4768617538708764
epoch:724
error:1.5384864165588616
epoch:725
error:1.6779036626778785
epoch:726
error:1.7178542913163002
epoch:727
error:1.6472963873504205
epoch:728
error:1.59297699329924
epoch:729
error:1.5827090323819535
epoch:730
error:1.5175852312924483
epoch:731
error:1.5994576161768272
epoch:732
error:1.6652064551665589
epoch:733
error:1.517147009924806
epoch:734
error:1.5468352047622898
epoch:735
error:1.5510465515847347
epoch:736
error:1.60522492521022
epoch:737
error:1.52836073940815
epoch:738
error:1.5974130568806597
epoch:739
error:1.5342147452094115
epoch:740
error:1.5811623873099936
epoch:741
error:1.573197496

epoch:950
error:1.4966453562115944
epoch:951
error:1.467708691279916
epoch:952
error:1.4266501126900064
epoch:953
error:1.4099023184831825
epoch:954
error:1.3882995052757114
epoch:955
error:1.408064473338454
epoch:956
error:1.4428417210745472
epoch:957
error:1.540368794469599
epoch:958
error:1.607283855008894
epoch:959
error:1.5144218257297057
epoch:960
error:1.6087434874777653
epoch:961
error:1.5485421362083602
epoch:962
error:1.568285627476148
epoch:963
error:1.4920178964869018
epoch:964
error:1.47660292690815
epoch:965
error:1.4726213502081986
epoch:966
error:1.4813685645415589
epoch:967
error:1.6503926779289295
epoch:968
error:1.4933949380457632
epoch:969
error:1.4704261375121281
epoch:970
error:1.65063508146022
epoch:971
error:1.5585924866591203
epoch:972
error:1.5359411005215071
epoch:973
error:1.5045186544914295
epoch:974
error:1.4953756986275064
epoch:975
error:1.505269394809185
epoch:976
error:1.4860688242844438
epoch:977
error:1.4605433376455368
epoch:978
error:1.493085103695

In [ ]:
y_valid = valid_labels#.cuda()
x_valid = valid_features.cuda()
net = torch.load('.//model_NN_master.t7')
with torch.no_grad():
    output_1 = net(x_valid)
output_1 = Tensor.cpu(output_1)  
out_1 = output_1.numpy()
error(out_1,y_valid)



In [ ]:
predict(x_test,use_gpu)


In [ ]:
from graphviz import Digraph
from torchviz import make_dot

In [19]:
make_dot(net(torch.randn(30,1)))


RuntimeError: size mismatch, m1: [30 x 1], m2: [30 x 30] at /opt/conda/conda-bld/pytorch_1549628766161/work/aten/src/TH/generic/THTensorMath.cpp:940